**_심층신경망(DNN) 기본기_**


# 5. 인공신경망 학습의 과제

### _Objective_

1. 선형회귀모델에서의 학습을 다시 살펴보며 `"모델의 학습"`이 어떤 의미인지 상기합니다.

2. 인공신경망이 학습되는 과정에서 `그래디언트(Gradient)`가 무엇인지 이해합니다.

3. 인공신경망의 학습에서 안고있는 과제를 이해하고 어떻게 해결할 수 있을지 생각해봅니다.

#### 필요한 패키지 호출

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import cm

In [ ]:
# 넘파이에서의 소숫점을 넷째자리수까지만 표기
np.set_printoptions(precision=5,suppress=True)

In [ ]:
# 시각화시 한글 폰트 설정을 위한 패키지
import matplotlib.font_manager as fm

In [ ]:
# plt 폰트 설정

fm._rebuild() # 폰트 관련 캐시 삭제
font_fname = '../../src/d2coding.ttf'
font_family = fm.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
plt.rcParams["font.size"] = 12

## [1. 선형회귀에서의 학습(Learning)]

---

pass

### 인공지능 모델의 학습

---

`인공지능 모델의 학습`은
1. 피쳐가 주어졌을 떄 모델이 목적을 잘 수행하도록 모델의 파라미터를 수정하는 것이다.
2. 학습 모델은 학습 알고리즘과 학습 데이터를 통해 스스로 파라미터를 수정한다.
3. 모델의 결과가 인간의 결과와의 차이가 더욱 적게 나는 방향으로 학습을 수행한다.

![6_1](./img/6_1.gif)

### [예제] 머신러닝의 기초, 선형회귀 모델에서의 학습

---

1. 예측값 $\hat{y} = \beta_1 x + \beta_0$ 의 가장 단순한 선형회귀 모델
2. 주어진 데이터(`x에 따른 y`)를 가장 잘 설명하는 $\beta_1$과 $\beta_0$를 찾기
3. `모델의 예측이 사람과 다른 정도`를 어떻게 나타내는지, 어떻게 줄여나가는지 이해

![5_1](./img/5_1.png)



### 문제 정의 : 주어진 데이터와 얻고자 하는 정보

---


+ 왓챠에서 제공하는 “보고싶어요” 수와 관객 수 간의 관계를 바탕으로 영화 “옥자”의 예상 관객 수 예측하기

![5_2](./img/5_2.png)

In [ ]:
movie_1 = pd.Series({"num_want_to_see": 8759,  "nums_audience": 487 }, name="마션")
movie_2 = pd.Series({"num_want_to_see": 10132, "nums_audience": 612 }, name="킹스맨")
movie_3 = pd.Series({"num_want_to_see": 12078, "nums_audience": 866 }, name="캡틴아메리카")
movie_4 = pd.Series({"num_want_to_see": 16430, "nums_audience": 1030}, name="인터스텔라")
movie_df = pd.DataFrame([movie_1, movie_2, movie_3, movie_4])
movie_df

### 선형회귀 모델링 : 입력값과 출력값 정의

---

실측값(Ground Truth)으로 입력값 $x$에 따른 예측값$\hat{y}$을 내는 선형회귀 모델을 생성

![5_3](./img/5_3.png)

In [ ]:
x = movie_df["num_want_to_see"] ##
y = movie_df["nums_audience"] ##
input_x = 12008 ## 입력값 x

In [ ]:
# 시각화로 확인하기

plt.figure(figsize=(8,8))
plt.title("영화 '보고싶어요 수'에 따른 '총 관객수'")
plt.xlabel(r"피쳐($x$): 보고싶어요 수(명)")
plt.ylabel(r"라벨($y$): 총 관객 수(만명)"); 

plt.scatter(x,y, label="실측값") ## x,y 만 지우고 타이핑
plt.axvline(input_x, lw=1, ls="--", c="C1", label="예측 입력값") ## input_x만 지우고 타이핑

plt.legend()
plt.show()

```python
"""
여기서, 
+ $x$를 피쳐(feature), $y$를 레이블(Label)
+ 주어진 데이터; $x,y$를 실측값(Ground Truth)라 한다.
"""
```

### 선형회귀 모델링 : 선형회귀 정의

---

`입력값`($x$; feature)에 따른 `출력값`($\hat{y}$; prediction value, 예측값)의 관계를 선형회귀로 정의하기

![5_4](./img/5_4.png)

#### 가중치 $a$, $b$에 따른 선형회귀 추세선 확인
모델링에 따라 다양한 가능한 가중치의 조합이 존재
* $a=0.05, b= 100$일때, $\hat y_{red} = 0.05x+100$
* $a=0.08, b=-240$일때, $\hat y_{green} = 0.08x-240$
* $a=0.12, b=-600$일때, $\hat y_{blue} = 0.12x-600$

In [ ]:
# 선형방정식 예시
model_red = lambda x: 0.05*x + 100
model_green = lambda x: 0.08*x - 240
model_blue = lambda x: 0.12*x - 600

![5_5](./img/5_5.png)

#### 각 회귀모델의 "총 관객수" 예측값

입력값($x$) = 영화 "옥자"의 "보고싶어요 수(명)" : 12008명

In [ ]:
input_x = 12008

y_blue = model_blue(input_x)
y_green = model_green(input_x)
y_red = model_red(input_x)

print(y_blue)
print(y_green)
print(y_red)

![5_7](./img/5_7.png)

```python
" ※ 이 때, 어떤 추세선이 더 잘 예측하는지 기준이 필요 "
```

### 손실값(Loss)과 손실함수(Loss Function)

---


1. "모델이 잘 예측한다." → "모델의 예측값과 실측값의 차이가 적다."


2. "예측값과 실측값에 차이가 나는 정도" 기준 → `손실(Loss)`


3. 예측값과 실측값으로 손실을 계산하는 함수 → `손실함수(Loss Function)`


![5_8](./img/5_8.png)

( ※ 손실값(Loss)는 더 넓은 의미로 비용(Cost) 라고도 부른다. <br>
이 때, 비용을 계산하는 함수를 비용함수(Cost Function)라고 한다.)

### 손실함수: 평균제곱오차(MSE; Mean Squared Error)

---

실측값 $y$와 예측값 $\hat{y}$에 대해<br>
모델의 예측값과 실측값의 차이 ($y - \hat{y}$) → `오차(Error)`

$$평균제곱오차\; MSE = \frac{1}{n}\sum_{i=1}^{n}( y_i - \hat{y}_i)^2$$

![5_9](./img/5_9.png)

#### 선형회귀 모델의 예측값 $\hat{y}$ 계산
실측값에서의 각 피쳐 $x$에 대한 모델의 예측값 $\hat{y} = w_1 x + w_0$

In [ ]:
# 피쳐 x에 대한 각 모델별 예측값

y_pred_blue = model_blue(x.values)
y_pred_red = model_red(x.values)
y_pred_green = model_green(x.values)

In [ ]:
pred_df = movie_df.assign(pred_blue = y_pred_blue, pred_red = y_pred_red, pred_green = y_pred_green)
pred_df

#### 각 예측값 $\hat{y}$ 의 오차 (Error) 계산
실측값에서의 각 라벨 $y$와 예측값 $\hat{y}$의 오차 $\text{Error} = y - \hat{y}$

In [ ]:
error_blue = y - y_pred_blue
error_red = y - y_pred_red
error_green = y - y_pred_green

In [ ]:
pred_df = pred_df.assign(error_blue = error_blue, error_red = error_red, error_green = error_green)
pred_df

#### 평균제곱오차(MSE) 계산
$\text{평균제곱오차} MSE = \frac{1}{n}\sum(Error)^2$

1. 오차 ($y-\hat{y}$) 는 음(-)의 값이 나올 수 있다. → 제곱하여 양수로 바꾸어 줌 : $(y-\hat{y})^2$

2. 다수의 데이터에 대한 모델의 오차 : 오차들의 합 SSE(Sum of Error) $\sum(y- \hat{y})^2$ <br>

3. SSE는 데이터가 많을수록 커진다. → 각 데이터의 오차를 평균 $\frac{1}{n}\sum(y-\hat{y})^2$<br>
(비교하는 데이터의 개수와는 별개로 손실값을 계산하고자 한다.)

In [ ]:
# MSE 계산
mse_blue = (error_blue**2).mean()
mse_red = (error_red**2).mean()
mse_green = (error_green**2).mean()

In [ ]:
print(mse_blue)
print(mse_red)
print(mse_green)

![5_10](./img/5_10.png)

```python
" ※ 목적 :  손실값 MSE를 최소화하는 최소화하는 모델을 찾는다. " : 최적화 
```

### 최적화(Optimization)

---
입력값 $x$, 함수의 파라미터 $w$, 함수의 결과 $f(x; w)$일때,<br> 

`최적화(Optimization)` : 함수 $f$의 결과 $f(x; w)$를 최대화 또는 최소화 하는 파라미터 $w$를 찾는 문제


( ⚠︎ 수학적 최적화 정의 [ko.wikipedia.org/wiki/수학적_최적화](https://ko.wikipedia.org/wiki/%EC%88%98%ED%95%99%EC%A0%81_%EC%B5%9C%EC%A0%81%ED%99%94) 에서 머신러닝 학습의 이해를 돕기 위한 설명 )

#### 파라미터와 함수의 정의
머신러닝의 학습에서 최적화의 대상이 되는 함수, 손실함수<br>
$\hat{y} = ax+b$ 일때, $\text{평균제곱오차} MSE = \frac{1}{n}\sum(\hat{y_i} - y_i)^2 = \frac{1}{n}\sum(ax_i+b - y_i)^2$

In [ ]:
mse = lambda x,y,a,b: ((a*x + b - y)**2).mean()

# 예컨데, a=0.1, b=-100에서
mse(x,y,a=0.1,b=-100)

이 때, 실측값($x,y$)는 정해져 있으므로 MSE는 ($a, b$)의 값에 따라 결정된다.

즉, 손실함수 Loss(x; w)에서 머신러닝 모델의 최적화 → 손실함수를 최소화 하는 w의 값을 찾기 $\underset{a, b}{\text{minimize}} \; Loss(a,b)$

![5_11](./img/5_11.png)

#### 최적의 a, b의 값을 찾는 방법, 그리드서치(?)

`그리드서치(Grid Search)`:  가능한 ($a, b$)의 조합으로 손실값을 계산하여 최소가 되는 ($a, b$) 찾기

In [ ]:
a_case = np.linspace(-1, 1, 100) # 100가지 경우
b_case = np.linspace(-1000, 1000, 100) # 100가지 경우

# 가능한 모든 a, b 조합
a_b_cases = np.stack(np.meshgrid(a_case,b_case), axis=-1).reshape(-1,2)

In [ ]:
# 모든 경우의 a, b에서 Loss 값 계산하기
losses = []
for a, b in a_b_cases:
    loss = mse(x,y,a,b)
    losses.append(loss)
losses = np.array(losses)

In [ ]:
# loss를 최소화 하는 a, b의 값
optim_a, optim_b = a_b_cases[np.argmin(losses)]
optim_y = optim_a * input_x + optim_b
print(f"손실을 최소화하는 최적의 a 값 : {optim_a:.5f}")
print(f"손실을 최소화하는 최적의 b 값 : {optim_b:.5f}")
print(f"a, b 파라미터의 선형회귀 모델로 예측한 옥자 총 관객수 : {optim_y:.2f}")

In [ ]:
# 시각화로 확인하기

x_extent = np.linspace(9000,16000)
y_linear = optim_a * x_extent + optim_b

plt.figure(figsize=(8,8))
plt.title("영화 '보고싶어요 수'에 따른 '총 관객수'")
plt.xlabel(r"피쳐($x$): 보고싶어요 수(명)")
plt.ylabel(r"라벨($y$): 총 관객 수(만명)"); 

plt.scatter(x,y, label="실측값")
plt.scatter(input_x, optim_y, c="C1", label="옥자 예측값")
plt.axvline(input_x, lw=1, ls="--", c="C1", label="예측 입력값")
plt.plot(x_extent,y_linear, label="선형회귀모델")

plt.legend()
plt.show()

In [ ]:
# 파라미터 a, b에 따른 Loss 값 시각화
mse_cases = np.concatenate([a_b_cases, losses.reshape(-1,1)], axis=-1)
mse_cases = mse_cases.reshape(100,100,3)

fig = plt.figure(figsize=(10,10))
ax = plt.gca(projection='3d')
ax.plot_surface(X=mse_cases[:,:,0],
                Y=mse_cases[:,:,1],
                Z=mse_cases[:,:,2],
                cmap=cm.coolwarm)
ax.set_xlabel('파라미터 a')
ax.set_ylabel('파라미터 b')
ax.set_zlabel('손실값 Loss')
plt.show()

```python
"※ 하지만, 파라미터의 모든 경우를 고려하기에는 연산량이 지나치게 많아진다." 
→ 수학적으로 최적화하는 알고리즘이 필요 : 최적화함수
```

In [ ]:
# 만약 파라미터의 수가 5개이고, 각각 100가지 경우만 고려하여 손실을 계산한다고 해도
100**5 # 총 100억가지 경우의 수가 발생

### 최적화함수(Optimizer)


---

최적화 함수  : 현재의 파라미터에서 더 최적화된 방향으로 파라미터를 갱신하는 방법<br>
대표적인 최적화 함수, `경사하강법(Gradient Descent; 그래디언트 디센트)` : 

$w := w - \gamma \frac{\partial Loss}{\partial w}$


( 위키피디아의 경사하강법 정의 [ko.wikipedia.org/wiki/경사_하강법](https://ko.wikipedia.org/wiki/%EA%B2%BD%EC%82%AC_%ED%95%98%EA%B0%95%EB%B2%95))

![5_12](./img/5_12.png)

![5_13](./img/5_13.png)

![5_14](./img/5_14.png)

![5_15](./img/5_15.png)

![5_16](./img/5_16.png)

![5_17](./img/5_17.png)

![5_18](./img/5_18.png)

![5_19](./img/5_19.png)

### 경사하강법 (Gradient Descent)

$m$개의 데이터에서 모델($ax + b$)의 손실(Loss)이<br>
$\text{평균제곱오차} MSE =  \frac{1}{m}\sum(ax^{(i)}+b - y^{(i)})^2$일때,

In [ ]:
model = lambda x,a,b: a*x+b
mse = lambda pred, true: ((pred - true)** 2).mean()

#### 무작위로 현재 파라미터를 결정

In [ ]:
# 무작위로 현재의 파라미터 a가 -0.75, b가 -100라고 한다면
a = -0.75
b = -100

# 현재의 Loss값
model(x,a,b)
y_pred = model(x,a,b)
mse(y_pred,y)
loss = mse(y_pred,y)

#### 그래디언트(Gradient)의 계산

$Loss =  \frac{1}{m}\sum(ax^{(i)}+b - y^{(i)})^2$일때, 파라미터 $a$, $b$의 그래디언트:<br>
$$\frac{\partial Loss}{\partial a} = \frac{1}{m}\sum_{i=1}^{m}(ax^{(i)}+b - y^{(i)})x^{(i)}$$<br>
$$\frac{\partial Loss}{\partial b} = \frac{1}{m}\sum_{i=1}^{m}(ax^{(i)}+b - y^{(i)})$$

( ※ 미분 법칙 중 "선형성", "곱의 법칙", "연쇄법칙" 참조 : [ko.wikipedia.org/wiki/미분법](https://ko.wikipedia.org/wiki/%EB%AF%B8%EB%B6%84%EB%B2%95) )<br>
( ⚠︎ 여기서, 그래디언트의 상수곱은 무시하였다. )

In [ ]:
derivative_a = lambda x,y,a,b : ((a*x + b - y)*x).mean()
derivative_b = lambda x,y,a,b : (a*x + b - y).mean()

In [ ]:
gradient_a = derivative_a(x,y,a,b)
gradient_b = derivative_b(x,y,a,b)

gradient_a, gradient_b

#### 러닝레이트($\gamma$; learning rate)

파라미터의 갱신(Update)에서,<br>
$a_{new} := a_{old} - \gamma \frac{\partial Loss}{\partial a}$ , 
$b_{new} := b_{old} - \gamma \frac{\partial Loss}{\partial b}$

러닝레이트 $\gamma$ 를 결정해야 하는 이유

In [ ]:
# 파라미터를 갱신하기 전

loss_old = mse(model(x,a,b), y)
print(f"갱신 전 손실 MSE : {loss_old}") # 손실값

In [ ]:
print(f"파라미터 a, b : {a}, {b}") # 파라미터
print(f"그래디언트 d_a, d_b : {gradient_a}, {gradient_b}") # 그래디언트
## 파라미터에 비해 그래디언트의 값이 지나치게 크다.

**러닝레이트가 0.001이라면**

In [ ]:
# 러닝레이트가 0.001 이라면
lr = 0.001
gradient_a * lr, gradient_b * lr ##

In [ ]:
# 파라미터 갱신
print(a, b) ##
a_new = a - lr * gradient_a ##
b_new = b - lr * gradient_b ##


a, b ##
a_new, b_new ##

In [ ]:
# 파라미터를 갱신한 후의 손실

loss_new = mse(model(x,a_new,b_new), y)
print(f"갱신 전 손실 MSE : {loss_old}")
print(f"갱신 후 손실 MSE : {loss_new}")
## 손실값이 되려 더 커졌다. 더욱 적절한 러닝레이트를 찾아야 한다.

```python
"※ 너무 큰 러닝레이트는 손실을 오히려 증가시킨다."
```

**러닝레이트가 1e-12 이라면**

In [ ]:
# 러닝레이트가 0.000000000001 이라면
lr = 1e-12
gradient_a * lr, gradient_b * lr ##

In [ ]:
# 파라미터 갱신
a_new = a - lr * gradient_a
b_new = b - lr * gradient_b

a, b ##
a_new, b_new

In [ ]:
# 파라미터를 갱신한 후

loss_new = mse(model(x,a_new,b_new), y)
print(f"갱신 전 손실 MSE : {loss_old}")
print(f"갱신 후 손실 MSE : {loss_new}")
## 손실값에 차이를 너무 조금밖에 주지 못했다.

```python
"※ 너무 작은 러닝레이트는 손실을 너무 적게 변화시킨다."
```

**러닝레이트가 2e-10 이라면**

In [ ]:
lr = 2e-10
gradient_a * lr, gradient_b * lr ##

In [ ]:
# 파라미터 갱신
a_new = a - lr * gradient_a
b_new = b - lr * gradient_b

a, b ##
a_new, b_new

In [ ]:
# 파라미터를 갱신한 후

loss_new = mse(model(x,a_new,b_new), y)
print(f"갱신 전 손실 MSE : {loss_old}")
print(f"갱신 후 손실 MSE : {loss_new}")

## 확실히 줄었다.

```python
"※ 실험을 통해 적절한 러닝레이트를 찾아야 함."
```

#### 에폭(epoch)의 결정

데이터셋 전체를 모델의 학습에 사용한 횟수 : `에폭(epoch)`

In [ ]:
# 만약, 데이터를 300번 사용하여 학습할 경우
epochs = 300

for epoch in range(epochs):
    # 모델 학습
    pass

### 학습 진행하기

---

지금까지 결정된 하이퍼파라미터(hyper parameter)를 통해 선형회귀 모델을 학습

실측값 : ($x, y$), 예측값 : $\hat{y}$, 파라미터 : $a, b$<br>
모델 : $\hat{y} = ax + b$<br>
손실함수 :  MSE = $\frac{1}{n}\sum_{i=1}^{n}( y_i - \hat{y}_i)^2$<br>
최적화함수 경사하강법 : $w := w - \gamma \frac{\partial Loss}{\partial w}$<br>
러닝레이트 : $\gamma$ = 2e-10<br>
에폭 : 300회

In [ ]:
# 학습 기록 
# 현재의 파라미터와 손실값으로 0번째 학습 기록
a_history = [a]
b_history = [b]
loss_history = [loss]

for epoch in range(epochs):
    
    # 갱신 전 파라미터
    a_old = a_history[-1]
    b_old = b_history[-1]
    
    # 예측값 도출(순전파)
    y_pred = model(x,a_old,b_old)
    
    # 그래디언트 계산
    gradient_a = ((y_pred - y)*x).mean()
    gradient_b = ((y_pred - y)).mean()
    
    # 경사하강법을 통한 파라미터 갱신
    a_new = a_old - lr * gradient_a
    b_new = b_old - lr * gradient_b
    
    # 현 시점의 손실값 계산
    loss_new = mse(y_pred,y)
    
    
    # 결과 저장
    a_history.append(a_new)
    b_history.append(b_new)
    loss_history.append(loss_new)

### 결과 확인하기

---

`a_history`, `b_history`, `loss_history` 시각화

In [ ]:
history_df = pd.DataFrame({
        "a":a_history,
        "b":b_history,
        "loss":loss_history
})

#### 손실값 확인을 통한 학습 진행 경과

In [ ]:
history_df

In [ ]:
history_df.plot(y='loss')
plt.show()

#### 학습에 따른 파라미터의 변화 시각화

In [ ]:
# 최종적으로 학습된 파라미터
print(f"a : {a_new}")
print(f"b : {b_new}")

In [ ]:
# 파라미터 a, b에 따른 Loss 값 시각화
mse_cases = np.concatenate([a_b_cases, losses.reshape(-1,1)], axis=-1)
mse_cases = mse_cases.reshape(100,100,3)

fig = plt.figure(figsize=(10,10))
ax = plt.gca(projection='3d')

ax.scatter(history_df["a"], 
           history_df["b"], 
           history_df["loss"],
           c='r', 
           label="파라미터 변화")

ax.plot_surface(X=mse_cases[:,:,0],
                Y=mse_cases[:,:,1],
                Z=mse_cases[:,:,2],
                cmap=cm.coolwarm,alpha=0.5)
ax.set_xlabel('파라미터 a')
ax.set_ylabel('파라미터 b')
ax.set_zlabel('손실값 Loss')
ax.legend()
plt.show()

#### 최적화된 모델을 통한 예측

![5_2](./img/5_2.png)

In [ ]:
movie_df

In [ ]:
y_new = input_x * a_new + b_new

print(f"학습된 파라미터 a : {a_new:.5f}")
print(f"학습된 파라미터 b : {b_new:.5f}")
print(f"학습된 선형회귀 모델로 예측한 옥자 총 관객수 : {y_new:.2f}")

In [ ]:
# 시각화로 확인하기

x_extent = np.linspace(9000,16000)
y_linear = a_new * x_extent + b_new

plt.figure(figsize=(8,8))
plt.title("영화 '보고싶어요 수'에 따른 '총 관객수'")
plt.xlabel(r"피쳐($x$): 보고싶어요 수(명)")
plt.ylabel(r"라벨($y$): 총 관객 수(만명)"); 

plt.scatter(x,y, label="실측값")
plt.scatter(input_x, y_new, c="C1", label="옥자 예측값")
plt.axvline(input_x, lw=1, ls="--", c="C1", label="예측 입력값")
plt.plot(x_extent,y_linear, label="학습된 선형회귀모델")

plt.legend()
plt.show()

## [2. 인공신경망에서의 학습(Learning)]

---

pass

### 인공신경망에서의 학습

---

모델의 순전파 결과 $\hat{y}$와 실측값 $y$로 계산된
손실값 $Loss$로 각 레이어에서의 가중치 $W^{[l]}, b^{[l]}$ 을 업데이트

In [ ]:
# 데이터 준비

from tensorflow.keras.utils import get_file

fpath = get_file("cancer_dataset.csv",
                 "https://s3.ap-northeast-2.amazonaws.com/pai-datasets/alai-deeplearning/cancer_dataset.csv")
cancer_df = pd.read_csv(fpath)

# 피쳐(feature) X, 라벨(label) y

X = cancer_df[['age','tumor_size']]
y = cancer_df["label"]

X = (X - X.min()) / (X.max() - X.min()) # 피쳐(feature) X의 min-max 정규화
y = y.values.reshape(-1,1) # 라벨 y를 입력 가능한 형태로 모양변환

#### 인공신경망의 순전파를 통한 예측

![5_23](./img/5_23.png)

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model

In [ ]:
inputs = Input(2)
dense_1 = Dense(3,"relu")(inputs)
dense_2 = Dense(3, "relu")(dense_1)
outputs = Dense(1, "sigmoid")(dense_2)

model = Model(inputs, outputs)

In [ ]:
model.predict(X) # 순전파 진행
y_pred = model.predict(X) # 순전파 진행

#### 예측값과 실측값에 의한 손실 Loss 값 계산

모델의 순전파 결과 $\hat{y}$와 실측값 $y$로 계산된
손실값 $Loss$

![5_20](./img/5_20.png)

In [ ]:
# 평균제곱오차 MSE
loss = ((y_pred - y)**2).mean()
loss

#### 인공신경망에서의 경사하강법

학습이 진행될 대상 : 각 레이어간 유닛의 가중치 $W, b$ 

![5_21](./img/5_21.png)

In [ ]:
# 각 레이어에서의 가중치 확인
model.layers[1].weights # 업데이트 대상
## 1, 2, 3 바꾸어가며 확인

#### 경사하강법에 필요한 그래디언트

각 가중치를 업데이트하기 위해 계산해야할 그래디언트 $\frac{\partial Loss}{\partial W}$

![5_22](./img/5_22.png)

In [ ]:
tf.Variable([1]).assign_sub([1])

In [ ]:
# 경사하강법
# 알아야 하는 값, gradient = ???

# for layer in model.layers[1:]:
#     layer.weight[0].assign_sub(lr * ???)
#     layer.weight[1].assign_sub(lr * ???)

### 그래디언트의 계산 ( 수식을 전개 했을 경우 ) 

---

그래디언트 $\frac{\partial Loss}{\partial W}$, $\frac{\partial Loss}{\partial b}$ 을 계산하는 미분 수식을 전개하여 필요한 연산 확인

#### 예시) $w^{[1]}_{11}$ 의 업데이트를 위해 필요한 그래디언트,  $\frac{\partial Loss}{\partial w^{[1]}_{11}}$ 의 계산

![5_24](./img/5_24.png)

#### 1. 미분의 연쇄 법칙을 활용한 합성함수의 미분 전개

![5_25](./img/5_25.png)

미분의 연쇄법칙 : [ko.wikipedia.org/wiki/연쇄_법칙](https://ko.wikipedia.org/wiki/%EC%97%B0%EC%87%84_%EB%B2%95%EC%B9%99) : $\frac{\partial y}{\partial x} = \frac{\partial y}{\partial u}\cdot \frac{\partial u}{\partial x}$

#### 2. 미분의 연쇄법칙을 활용한 선형결합 미분 전개

다음 레이어의 각 유닛에서의 선형결합 $z_i^{[l]} = w_{1i}^{[l+1]}a_1^{[l]} + w_{2i}^{[l+1]}a_2^{[l]} + w_{3i}^{[l+1]}a_3^{[l]} + b_i^{[l+1]}$에 대한 미분 전개<br>



![5_26](./img/5_26.png)


다변수함수의 연쇄법칙 [wikipedia.org/wiki/연쇄_법칙](https://ko.wikipedia.org/wiki/%EC%97%B0%EC%87%84_%EB%B2%95%EC%B9%99) : $\frac{\partial f}{\partial x_j} = \frac{\partial f}{\partial u_1}\frac{\partial u_1}{\partial x_j} + \frac{\partial f}{\partial u_2}\frac{\partial u_2}{\partial x_j} + ... + \frac{\partial f}{\partial u_m}\frac{\partial u_m}{\partial x_j}$<br>

![5_27](./img/5_27.png)

#### 3. 동일한 방법으로 마지막 레이어까지 연쇄법칙 적용

![5_28](./img/5_28.png)

### 인공신경망 학습의 과제 

---

인공신경망의 학습에 필요한 각 가중치에서의 그래디언트의 문제점과 해결 아이디어

#### 복잡한 그래디언트 계산

![5_29](./img/5_29.png)

#### 역전파의 아이디어

![5_30](./img/5_30.png)

$w^{[1]}_{11}$의 그래디언트 계산에는 $w_{31}^{[3]}$의 값과 $w_{31}^{[3]}$의 그래디언트 계산에 필요한 연산이 포함

#### 역전파의 아이디어
![5_31](./img/5_31.png)


전방 레이어(입력층 방향)의 가중치에 대한 그래디언트 계산 수식에는 <br>
후방 레이어(출력층 방향)의 가중치에 대한 그래디언트 계산 수식에서 중복으로 연산

#### 역전파의 아이디어

이미 계산된 미분값을 저장한 후 이를 재사용


1. 순전파 진행시, 손실값이 필요하지 않은 미분값을 미리 계산 후 저장
2. 순전파를 통해 계산된 예측값과 실측값으로 손실값 Loss 계산
3. 출력에서 입력 방향으로 Loss에 대한 미분값을 연산하며 필요한 그래디언트 계산


![5_32](./img/5_32.png)

## `5. 인공신경망 학습의 과제 ` 마무리


---

#### references

+ Deep Learning - Ian Goodfellow [deeplearningbook.org/](https://www.deeplearningbook.org/)

![](../../src/logo.png)

# \[부록 컨텐츠\]

---

pass

#### reference

+ 넘파이(numpy)의 `np` 별칭
  + [numpy Stylistic Guidelines](https://docs.scipy.org/doc/numpy/dev/#stylistic-guidelines)
  
 
+ 블로그 글 : [medium.com/@erikhallstrm/backpropagation-from-the-beginning-77356edf427d](https://medium.com/@erikhallstrm/backpropagation-from-the-beginning-77356edf427d)

---

---
Pass

### 모델이 학습되면서 변화를 시각화하기

In [ ]:
# 시각화시 한글 폰트 설정을 위한 패키지
import matplotlib.font_manager as fm
# 시각화시 한글 폰트 설정을 위한 패키지
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import HTML
from IPython.display import display

tag = HTML('''<script>
code_show=true; 
function code_toggle() {
    if (code_show){
        $('div.cell.code_cell.rendered.selected div.input').hide();
    } else {
        $('div.cell.code_cell.rendered.selected div.input').show();
    }
    code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<a href="javascript:code_toggle()">폰트설정 코드</a>''')
display(tag)


# plt 폰트 설정

fm._rebuild() # 폰트 관련 캐시 삭제
font_fname = '../../src/d2coding.ttf'
font_family = fm.FontProperties(fname=font_fname).get_name()

plt.rcParams["font.family"] = font_family
plt.rcParams["font.size"] = 12

#### 데이터 만들기

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

In [ ]:
np.random.seed(100)
tf.random.set_seed(100)
X1_1 = np.random.normal(25, 8, 1000)
X1_2 = np.random.normal(80, 10, 1000)
X1_3 = np.random.normal(50, 10, 1000)

In [ ]:
X2_1 = np.random.normal(30, 8, 1000)
X2_2 = np.random.normal(40, 10, 1000)
X2_3 = np.random.normal(70, 10, 1000)

In [ ]:
X1_1 = np.where(X1_1 > 100, 100, X1_1)
X1_2 = np.where(X1_2 > 100, 100, X1_2)
X1_3 = np.where(X1_3 > 100, 100, X1_3)

X2_1 = np.where(X2_1 > 100, 100, X2_1)
X2_2 = np.where(X2_2 > 100, 100, X2_2)
X2_3 = np.where(X2_3 > 100, 100, X2_3)

X1_1 = np.where(X1_1 <= 0, 0, X1_1)
X1_2 = np.where(X1_2 <= 0, 0, X1_2)
X1_3 = np.where(X1_3 <= 0, 0, X1_3)

X2_1 = np.where(X2_1 <= 0, 0, X2_1)
X2_2 = np.where(X2_2 <= 0, 0, X2_2)
X2_3 = np.where(X2_3 <= 0, 0, X2_3)

In [ ]:
Y_1 = np.full_like(X1_1, 0)
Y_2 = np.full_like(X1_2, 1)
Y_3 = np.full_like(X1_3, 2)

In [ ]:
X1 = np.concatenate([X1_1, X1_2, X1_3])
X2 = np.concatenate([X2_1, X2_2, X2_3])
Y = np.concatenate([Y_1, Y_2, Y_3])

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")
plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("분류할 데이터")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

In [ ]:
X1_max = X1.max()
X1_min = X1.min()
X2_max = X2.max()
X2_min = X2.min()

In [ ]:
X1_p = (X1 - X1_min) / (X1_max - X1_min)
X2_p = (X2 - X2_min) / (X2_max - X2_min)

In [ ]:
X = np.stack([X1_p, X2_p], axis=-1)

In [ ]:
Y_ohe = to_categorical(Y)

#### 분류모델 만들기

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import SGD

In [ ]:
inputs = Input(2)
dense_1 = Dense(40,"relu")(inputs)

dense_2 = Dense(40,"relu")(dense_1)

outputs = Dense(3, "softmax")(dense_2)

In [ ]:
model = Model(inputs, outputs)

In [ ]:
loss = CategoricalCrossentropy()
optim = SGD()

In [ ]:
model.compile(optim, loss)

In [ ]:
model.summary()

#### 0회 학습시 시각화

In [ ]:
X1_range = np.arange(0, 100)
X1_range_p = (X1_range - X1_min)/(X1_max - X1_min)

X2_range = np.arange(0, 100)
X2_range_p = (X2_range - X2_min)/(X2_max - X2_min)
X1_mesh, X2_mesh = np.meshgrid(X1_range_p, X2_range_p)

In [ ]:
X_mesh = np.stack([X1_mesh, X2_mesh], axis=-1).reshape(-1, 2)

In [ ]:
Y_pred = model.predict(X_mesh)
Y_pred = np.argmax(Y_pred, axis=-1)
Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("학습이 진행되지 않은 인공신경망 모델의 분류 예측")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

#### n 회 학습시 시각화

In [ ]:
n_learning=20

for n in range(n_learning):
    n = n + 1
    model.fit(X,Y_ohe,
             epochs=1,
             validation_split=0.1)
    Y_pred = model.predict(X_mesh)
    Y_pred = np.argmax(Y_pred, axis=-1)
    Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)
    plt.figure(figsize=(8, 6))
    plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
    plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
    plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

    plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

    plt.xticks(np.arange(0, 110, 10), fontsize=12)
    plt.yticks(np.arange(0, 110, 10), fontsize=12)
    plt.title(f"학습이 {n}회 진행된 인공신경망 모델의 분류 예측")
    plt.xlim([0, 100])
    plt.ylim([0, 100])
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"./img/6_img/{n}.png",
                       bbox_inces='tight',
                       pad_inches=0, )

#### 1회 학습시 시각화

In [ ]:
model.fit(X,Y_ohe,
         epochs=1,
         validation_split=0.1)

In [ ]:
Y_pred = model.predict(X_mesh)
Y_pred = np.argmax(Y_pred, axis=-1)
Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("학습이 1회 진행된 인공신경망 모델의 분류 예측")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

#### 2회 학습시 시각화

In [ ]:
model.fit(X,Y_ohe,
         epochs=1,
         validation_split=0.1)

In [ ]:
Y_pred = model.predict(X_mesh)
Y_pred = np.argmax(Y_pred, axis=-1)
Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("학습이 2회 진행된 인공신경망 모델의 분류 예측")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

#### 5회 학습시 시각화

In [ ]:
model.fit(X,Y_ohe,
         epochs=3,
         validation_split=0.1)

In [ ]:
Y_pred = model.predict(X_mesh)
Y_pred = np.argmax(Y_pred, axis=-1)
Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("학습이 5회 진행된 인공신경망 모델의 분류 예측")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

#### 10회 학습시 시각화

In [ ]:
model.fit(X,Y_ohe,
         epochs=5,
         validation_split=0.1)

In [ ]:
Y_pred = model.predict(X_mesh)
Y_pred = np.argmax(Y_pred, axis=-1)
Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("학습이 10회 진행된 인공신경망 모델의 분류 예측")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

#### 20회 학습시 시각화

In [ ]:
model.fit(X,Y_ohe,
         epochs=10,
         validation_split=0.1)

In [ ]:
Y_pred = model.predict(X_mesh)
Y_pred = np.argmax(Y_pred, axis=-1)
Y_pred = Y_pred.reshape(X1_range.size,X2_range.size)

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(X1_1,X2_1,c = "C0",s=3, alpha=0.5, label="a")
plt.scatter(X1_2,X2_2,c = "C1",s=3, alpha=0.5, label="b")
plt.scatter(X1_3,X2_3,c = "C2",s=3, alpha=0.5, label="c")

plt.contourf(X1_range, X2_range, Y_pred, alpha= 0.2, cmap ="Set3")

plt.xticks(np.arange(0, 110, 10), fontsize=12)
plt.yticks(np.arange(0, 110, 10), fontsize=12)
plt.title("학습이 20회 진행된 인공신경망 모델의 분류 예측")
plt.xlim([0, 100])
plt.ylim([0, 100])
plt.legend()
plt.show()

In [ ]:
x = np.random.uniform(0, 10, 30)
y_lr = 0.5*x + 2
y = y_lr + np.random.normal(0,0.5,size=30)

(이때, 각 유닛에서의 선형결합은 $a_1^{[l]}$에 대해서 역함수에 해당한다.)<br>

역함수정리 : $\frac{\partial y}{\partial x} = \frac{1}{\frac{\partial x}{\partial y}}$

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(x,y, c="C0", label="data")
plt.plot(x,y_lr, c="C1", label=r"$\hat{y} = β_1x + β_0$")

plt.xlim(0,10)
plt.ylim(0,10)
plt.legend(loc=2)

plt.show()

In [ ]:
x_lr = np.arange(8000, 18000) # 추세선을 그릴 x의 범위

y_red = 0.05 * x_lr + 100
y_green = 0.08 * x_lr - 240
y_blue = 0.12 * x_lr - 600

In [ ]:
# 시각화로 확인하기

plt.figure(figsize=(8,8))
plt.title("영화 '보고싶어요 수'에 따른 '총 관객수'")
plt.xlabel(r"피쳐($x$): 보고싶어요 수(명)")
plt.ylabel(r"라벨($y$): 총 관객 수(만명)"); 

plt.scatter(x,y)

plt.plot(x_lr, y_red,"r--", label=r"$\hat y_{red} = 0.05x+100$")
plt.plot(x_lr, y_green,"g--", label=r"$\hat y_{green} = 0.08x-240$")
plt.plot(x_lr, y_blue,"b--", label=r"$\hat y_{blue} = 0.12x-660$")


plt.legend()
plt.show()

In [ ]:
# 선형방정식 예시
model_red = lambda x: 0.05*x + 100
model_green = lambda x: 0.08*x - 240
model_blue = lambda x: 0.12*x - 600

In [ ]:
# 시각화로 확인하기

plt.figure(figsize=(8,8))
plt.title("영화 '보고싶어요 수'에 따른 '총 관객수'")
plt.xlabel(r"피쳐($x$): 보고싶어요 수(명)")
plt.ylabel(r"라벨($y$): 총 관객 수(만명)"); 

plt.scatter(x,y)

plt.plot(x_lr, y_red,"r--", label=r"$\hat y_{red} = 0.05x+100$")
plt.plot(x_lr, y_green,"g--", label=r"$\hat y_{green} = 0.08x-240$")
plt.plot(x_lr, y_blue,"b--", label=r"$\hat y_{blue} = 0.12x-660$")

plt.axvline(input_x, lw=1, ls="--", c="C1", label="예측 입력값")

plt.legend()
plt.show()

In [ ]:
# 각 선형방정식을 통한 입력과 출력
x_lr = np.array([8000, 18000])
y_red = model_red(x_lr)
y_green = model_green(x_lr)
y_blue = model_blue(x_lr)

In [ ]:
# 시각화로 확인하기

plt.figure(figsize=(8,8))
plt.scatter(x,y)

plt.plot(x_lr, y_red,"r--")
plt.plot(x_lr, y_green,"g--")
plt.plot(x_lr, y_blue,"b--")
plt.show()